In [6]:
import os
import pandas as pd
import pycep_correios
from geopy.geocoders import Nominatim, HereV7, TomTom
from geopy.extra.rate_limiter import RateLimiter

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

uf = 'AC'
file_path = os.path.join(f'postos_saude_brasil_{uf}.csv')
df = pd.read_csv(file_path)
df.head()

,UF,MUNICIPIO,ESTABELECIMENTO,LOGRADOURO,NUMERO,BAIRRO,ENDERECO_COMPLETO,latitude,longitude,address,point
0,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA RICARDO MONTEIRO ROLA,RUA 07 QUEDAS,284,CENTRO,"RUA 07 QUEDAS, CENTRO, ACRELANDIA, AC, BRASIL",-10.07561,-67.05003,"Rua Sete Quedas, Acrelândia","10 4m 32.196s S, 67 3m 0.108s W"
1,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA JOAO DANIEL DAMASCENO,BR 364 KM 114,114,ZONA RURAL,"BR 364 KM 114, ZONA RURAL, ACRELANDIA, AC, BRASIL",-22.28388,-51.91008,"Via de Acesso à Zona Rural, 114, 19260-000, Mi...","22 17m 1.968s S, 51 54m 36.288s W"
2,AC,ACRELANDIA,UNIDADE MISTA DE SAUDE DE ACRELANDIA,AVENIDA PARANA,346,CENTRO,"AVENIDA PARANA, CENTRO, ACRELANDIA, AC, BRASIL",-10.07684,-67.05564,"Avenida Paraná, 69945, Acrelândia","10 4m 36.624s S, 67 3m 20.304s W"
3,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA CICERO BATISTA,BR 364 KM 90 RAMAL GRANADA KM 16,16,ZONA RURAL,"BR 364 KM 90 RAMAL GRANADA KM 16, ZONA RURAL, ...",-17.34786,-49.93093,"Via de Acesso à Zona Rural, 90, 75940-000, Edéia","17 20m 52.296s S, 49 55m 51.348s W"
4,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA REDENCAO,AC 475 VILA REDENCAOI,NaN,ZONA RURAL,"AC 475 VILA REDENCAOI, ZONA RURAL, ACRELANDIA,...",-22.28064,-51.91064,"Via de Acesso à Zona Rural, 475, 19260-000, Mi...","22 16m 50.304s S, 51 54m 38.304s W"


In [7]:
geolocator_nominatim = Nominatim(user_agent="info_vacinas")
geolocator_tomtom = TomTom(api_key='wvuAFmBut64DLjQqey19XtfuMXZilzbj', user_agent="info_vacinas")
geolocator_here = HereV7(apikey=os.environ.get('HERE_API_KEY'), user_agent="info_vacinas")
geocode_nominatim = RateLimiter(geolocator_nominatim.geocode, min_delay_seconds=0)
geocode_tomtom = RateLimiter(geolocator_tomtom.geocode, min_delay_seconds=0)
geocode_here = RateLimiter(geolocator_here.geocode, min_delay_seconds=0)
tamanho = len(df)

print(f'Processando {str(tamanho).zfill(5)} registros')
for index, row in df[::-1].iterrows():
  print(f'Processando {str(index+1).zfill(5)} de {str(tamanho).zfill(5)}')
  # pula os registros que já possuem dados de localização
  if not row['latitude']:
    continue
  
  # inicializa a variável que receberá o retorno das APIs de localização
  location = None

  try:
    # tenta fazer a localização usando o Nominatim
    location = geolocator_nominatim.geocode(row['ENDERECO_COMPLETO'])
  except Exception as e:
    print(f'Erro: {e}')
    # se a localização não retorna nada, tenta pegar a localização no TomTom
    try:
      location = geolocator_tomtom.geocode(row['ENDERECO_COMPLETO'])
    except Exception as e:
      # se a localização não retorna nada, tenta pegar a localização no Here
      print(f'Erro: {e}')
      try:
        location = geolocator_here.geocode(row['ENDERECO_COMPLETO'])
      except Exception as e:
        # salva o CSV com os dados buscados até aquele momento
        df.to_csv(file_path, index=False)
        if not location:
          continue

  # se em nenhuma das três tentativas conseguiu retornar a localização, 
  # vai para o próximo registro
  if not location:
    continue

  print(location)

  # salva de 10 em 10 registros o arquivo de saída
  if index%50 == 0:
    df.to_csv(file_path, index=False)

  row['latitude'] = location.latitude
  row['longitude'] = location.longitude
  row['address'] = location.address
  row['point'] = location.point

df.to_csv(file_path, index=False)
df.head()

Processando 00243 registros
Rua Capitao Hipolito, Bairro Triangulo, Tarauacá, Região Geográfica Imediata de Tarauacá, Região Geográfica Intermediária de Cruzeiro do Sul, Acre, Região Norte, 69970-000, Brasil
Avenida Avelino Leal, Copacabana, Tarauacá, Região Geográfica Imediata de Tarauacá, Região Geográfica Intermediária de Cruzeiro do Sul, Acre, Região Norte, 69970-000, Brasil
Rua Antônio Pinto, Bairro Senador Pompeu, Tarauacá, Região Geográfica Imediata de Tarauacá, Região Geográfica Intermediária de Cruzeiro do Sul, Acre, Região Norte, 69970-000, Brasil
Rua Rio Grande do Sul, Vila Preventório, Centro, Rio Branco, Região Geográfica Imediata de Rio Branco, Região Geográfica Intermediária de Rio Branco, Acre, Região Norte, 69911-304, Brasil
Estrada do Calafate, Calafate, Rio Branco, Região Geográfica Imediata de Rio Branco, Região Geográfica Intermediária de Rio Branco, Acre, Região Norte, 69920-900, Brasil
Estrada da Sobral, Sobral, Centro, Rio Branco, Região Geográfica Imediata de R

In [ ]:
df.to_csv(file_path, index=False)
df.tail()